In [63]:
import tensorflow as tf
import numpy as np

In [64]:
# Read data from CSV. 
# Reference: 
# https://www.tensorflow.org/programmers_guide/reading_data
# https://stackoverflow.com/questions/37091899/how-to-actually-read-csv-data-in-tensorflow
# https://stackoverflow.com/questions/40143019/how-to-correctly-read-data-from-csvs-into-tensorflow

def read_csv_format(filename_queue):
    reader = tf.TextLineReader(skip_header_lines = 0)
    _, value = reader.read(filename_queue)
    default_values = np.full([9, 1], 0.).tolist()
    d = tf.decode_csv(value, default_values)
    return tf.stack(d[0:8]), tf.reshape(d[8], [1])

def input_pipeline(filenames, batch_size, num_epochs = None):
    filename_queue = tf.train.string_input_producer(filenames, num_epochs=num_epochs, shuffle=False)
    predictor, response = read_csv_format(filename_queue)
    min_after_dequeue = 10000
    capacity = min_after_dequeue + 3 * batch_size
    return tf.train.shuffle_batch(
        [predictor, response], batch_size = batch_size, capacity=capacity,
        min_after_dequeue = min_after_dequeue)    

In [71]:
x_tr, y_tr = input_pipeline(["pima-indians-diabetes.csv"], 1, 100)


In [72]:
print(x_tr)
print(y_tr)

Tensor("shuffle_batch_6:0", shape=(1, 8), dtype=float32)
Tensor("shuffle_batch_6:1", shape=(1, 1), dtype=float32)


In [77]:
x = tf.placeholder(tf.float32, shape=[None, 8])
y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_uniform([8, 1], -1.0, 1.0), name = 'weight')
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name = 'bias')

# W = tf.Variable(tf.random_normal([8, 1]), name = 'weights')
# b = tf.Variable(tf.random_normal([1], name='bias'))

In [74]:
def clip(x, vmin = 1e-7, vmax = 1-1e-7):
    return tf.clip_by_value(x, clip_value_max= vmax, clip_value_min= vmin)

In [78]:
# y_h = tf.sigmoid(tf.matmul(x, W) + b)
y_h = tf.sigmoid(tf.matmul(x, W) + b)

cost = - tf.reduce_mean(y * tf.log(y_h) + (1 - y) * tf.log(1 - y_h))
# cost = - tf.reduce_mean(y * tf.log(clip(y_h)) + (1 - y) * tf.log(clip(1 - y_h)))
# cost = -tf.reduce_mean(Y * tf.log(clip(hypothesis)) + (1 - Y) * tf.log(clip(1 - hypothesis)))
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

In [79]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    step = 1
    
    try:
        while not coord.should_stop():
            step += 1
            x_batch, y_batch = sess.run([x_tr, y_tr])
            cost_val, _ = sess.run([cost, train], feed_dict={x: x_batch, y: y_batch})
            if step % 70 == 0:
                print(step, cost_val)
                
    except tf.errors.OutOfRangeError:
        print('Done training, epoch reached')
        
    finally:
        coord.request_stop()
        
    coord.join(threads)

70 nan
140 nan
210 nan
280 nan
350 nan
420 nan
490 nan
560 nan
630 nan
700 nan
770 nan
840 nan
910 nan
980 nan
1050 nan
1120 nan
1190 nan
1260 nan
1330 nan
1400 nan
1470 nan
1540 nan
1610 nan
1680 nan
1750 nan
1820 nan
1890 nan
1960 nan
2030 nan
2100 nan
2170 nan
2240 nan
2310 nan
2380 nan
2450 nan
2520 nan
2590 nan
2660 nan
2730 nan
2800 nan
2870 nan


KeyboardInterrupt: 